In [130]:
import time

In [131]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for line in file:
            
            nodes = line.strip().split() 
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            # Dodavanje suseda
            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph

In [132]:
def greedy_burning(graf):
    
    S=[] # sekvenca zapaljenih cvoreva
    B= set() # skup zapaljenih cvoreva
    NB= set(graf.keys()) # skup nezapaljenih cvoreva, inicajlno su to svi cvorevi grafa
    
    # korak 1: poceti sa cvorom sa najvecim brojem susjeda, dodajemo ga u S i B i brisemo iz NB
    trenutni_cvor = max(graf, key=lambda node: len(graf[node]))
    S.append(trenutni_cvor)
    B.add(trenutni_cvor)
    NB.remove(trenutni_cvor)
    
    while len(B)<len(graf):
       
        # korak 2: zapaliti sve susjede (koji nisu zapaljeni) od cvorova u skupu zapaljenih, dodajemo ih u B, brisemo iz NB
        novozapaljeni = []
        for cvor in B:
            susjedi = graf[cvor]  # svi susjedi trenutnog zapaljenog cvora
            for susjed in susjedi:
                if susjed in NB:  # ako susjed nije zapaljen
                    novozapaljeni.append(susjed)  # dodajemo ga u zapaljene
                    NB.remove(susjed)  
        
        for nz in novozapaljeni:
            B.add(nz) #dodajemo sve susjede u zapaljene
       
        # korak 3: pronaci iduci cvor koji ce biti direktno zapaljen, dodati ga u S, B i izbrisati iz NB
        
        max_broj_zapaljenih = -1
        najbolji_cvor = None
        
        for cvor in NB:
            nezapaljeni_susjedi = len([n for n in graf[cvor] if n not in B])
            if nezapaljeni_susjedi>max_broj_zapaljenih:
                max_broj_zapaljenih = nezapaljeni_susjedi
                najbolji_cvor = cvor
        
        S.append(najbolji_cvor)
        B.add(najbolji_cvor)
        NB.remove(najbolji_cvor)
        
        
        '''
        print("------")
        print(S)
        print(B)
        print(NB)
        print(len(B))
        print(len(NB))'''
        
        
    return S
        

In [134]:
startTime = time.time()
#filename = "graf1.mtx"  
filename = "grafovi/dolphins.mtx"

graph = load_graph_from_mtx(filename)
burn_order = greedy_burning(graph)

print("Redoslijed paljenja čvorova:", burn_order)
print("Size",len(burn_order))
print("--- %s sekundi ---" % (time.time() - startTime))


Redoslijed paljenja čvorova: [15, 18, 20, 61]
Size 4
--- 0.001001119613647461 sekundi ---
